- Default PSO parameters
- max_back=20

In [1]:
from joblib import Parallel, delayed
import multiprocessing

In [2]:
import random

import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

import tengp
import symreg
from gpbenchmarks import get_data

from experiment_settings import nguyen7_funset, c1nguyen7_funset, pagie_funset, keijzer_funset, korns12_funset, vlad_funset

def plot_results(name, outs):
    print(name)
    costs = np.array(outs)
    print('min:', np.min(costs))
    print('mean:', np.mean(costs))
    print('median:', np.median(costs))
    print('variance:',np.var(costs))
    plt.figure()
    plt.title(name)
    sns.distplot(costs, kde=False);
    

def run_experiment(data_item, cost_function):
    results = []
    genes = []
    
    name, (x_train, y_train, x_test, y_test), params = data_item
    print(name)
    
    bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]
    
    pg.set_global_rng_seed(42)
    
    for i in range(10):
        fit, gs = run_experiment_instance(cost_function, x_train, y_train, params, bounds)
        results.append(fit)
        genes.append(gs)
    
    return results, genes


def run_experiment_instance(i, cost_function, x_train, y_train, params, bounds):
    print(i)
    prob = pg.problem(cost_function(x_train, y_train, params, bounds))
    algo = pg.algorithm(pg.pso(gen=2000))
    algo.set_verbosity(1)
    pop = pg.population(prob, 50)
    pop = algo.evolve(pop)
    return pop.champion_f[0], pop.champion_x
    

In [3]:
import pygmo as pg

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        
        pred = individual.transform(self.X)
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        return self.bounds

In [4]:
kw_params = {'real_valued': True, 'max_back': 20}

def constant1(x, _):
    return np.tile(1, x.shape)

pagie_funset.add(constant1, 2)

params_nguyen4 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_nguyen7 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_nguyen10 = tengp.Parameters(3, 1, 1, 50, nguyen7_funset, **kw_params)
params_pagie1  = tengp.Parameters(2, 1, 1, 50, pagie_funset, **kw_params)
params_keijzer6 = tengp.Parameters(2, 1, 1, 50, keijzer_funset, **kw_params)
params_korns = tengp.Parameters(6, 1, 1, 50, korns12_funset, **kw_params)
params_vlad = tengp.Parameters(6, 1, 1, 50, vlad_funset, **kw_params)

all_params = [params_nguyen7, params_pagie1, params_keijzer6, params_korns, params_vlad]

In [5]:
import random 

data = [
    ('nguyen4', symreg.get_benchmark_poly(random, 6), params_nguyen4),
    ('nguyen7', symreg.get_benchmark_nguyen7(random, None), params_nguyen7),
    ('nguyen10', symreg.get_benchmark_nguyen10(random, None), params_nguyen10),
    ('pagie1', symreg.get_benchmark_pagie1(random, None), params_pagie1),
    ('keijzer6', symreg.get_benchmark_keijzer(random, 6), params_keijzer6),
    ('korns12', symreg.get_benchmark_korns(random, 12), params_korns),
    ('vladislasleva4', symreg.get_benchmark_vladislasleva4(random, None), params_vlad)
]

In [6]:
num_cores = multiprocessing.cpu_count()
num_cores

4

# Nguyen 4

In [ ]:
%%time

ng4_res, ng4_genes = run_experiment(data[0], cost_function)

In [ ]:
%%time

name, (x_train, y_train, x_test, y_test), params = data[0]
print(name)

bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

results = Parallel(n_jobs=num_cores)(delayed(run_experiment_instance)(_, cost_function, x_train, y_train, params, bounds)
                                    for _ in range(4))



In [ ]:
import pickle

pickle.dump(ng4_res, open('ng4_res_100_000evals', 'wb'))
pickle.dump(ng4_genes, open('ng4_genes_100_000evals', 'wb'))

In [ ]:
plot_results('nguyen4', ng4_res)

# Nguyen 7


In [ ]:
%%time

ng7_res, ng7_genes = run_experiment(data[1], cost_function)

In [ ]:
%%time

name, (x_train, y_train, x_test, y_test), params = data[1]
print(name)

bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

results = Parallel(n_jobs=num_cores)(delayed(run_experiment_instance)(_, cost_function, x_train, y_train, params, bounds)
                                    for _ in range(20))

In [ ]:
ng7_res = [x[0] for x in results]
ng7_genes = [x[1] for x in results]

In [ ]:
import pickle

pickle.dump([x[0] for x in results], open('ng7_res_100_000evals', 'wb'))
pickle.dump([x[1] for x in results], open('ng7_genes_100_000evals', 'wb'))

In [ ]:
plot_results('nguyen7', ng7_res)

# Nguyen 10

In [ ]:
%%time

ng10_res, ng10_genes = run_experiment(data[2], cost_function)

In [ ]:
pickle.dump(ng10_res, open('ng10_res_100_000evals', 'wb'))
pickle.dump(ng10_genes, open('ng10_genes_100_000evals', 'wb'))

In [ ]:
plot_results(data[2][0], ng10_res)

# Pagie

In [ ]:
%%time

pag1_res, pag1_genes = run_experiment(data[3], cost_function)

In [ ]:
y_train

In [7]:
%%time

name, (x_train, y_train, x_test, y_test), params = data[3]
print(name)

bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

results = Parallel(n_jobs=num_cores)(delayed(run_experiment_instance)(_, cost_function, x_train, y_train, params, bounds)
                                    for _ in range(20))

pagie1
0
1
2
3


/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:110: RuntimeWarning: invalid value encountered in power
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:35: RuntimeWarning: invalid value encountered in sqrt
  return  np.sqrt(x)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: invalid value encountered in greater
  return np.log(x, out=np.copy(x), where=x>0)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:110: RuntimeWarning: invalid value encountered in power
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:109: RuntimeWarning: invalid value encountered in power
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:11: RuntimeWarning: in

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:109: RuntimeWarning: invalid value encountered in add
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: invalid value encountered in add
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: invalid value encountered in add
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:109: RuntimeWarning: invalid value encountered in add
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:107: RuntimeWarning: invalid value encountered in multiply
  U = (1-c)*x_l2 + c*x_u2
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: invalid value encountered in multiply
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: invalid value encountered in add
  L = (1-b)*x_l1 + b*x_u1
/mnt/c/Users/jarinov-notas/dev/tengp/te

/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:41: RuntimeWarning: overflow encountered in reciprocal
  return x**(-1)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:41: RuntimeWarning: overflow encountered in reciprocal
  return x**(-1)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:41: RuntimeWarning: overflow encountered in reciprocal
  return x**(-1)
/mnt/c/Users/jarinov-notas/dev/cgp-optimization-experiment/experiment_settings.py:41: RuntimeWarning: overflow encountered in reciprocal
  return x**(-1)


4
5
6
7


...

8
9
10
11


/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:110: RuntimeWarning: overflow encountered in add
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:110: RuntimeWarning: overflow encountered in add
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:110: RuntimeWarning: overflow encountered in add
  u = a*f_u(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:110: RuntimeWarning: overflow encountered in add
  u = a*f_u(L, U)


12
13
14
15


/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:132: RuntimeWarning: overflow encountered in add
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:132: RuntimeWarning: overflow encountered in add
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:132: RuntimeWarning: overflow encountered in add
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:132: RuntimeWarning: overflow encountered in add
  value = (1-coeff)*self.nodes[lower].value + coeff*self.nodes[upper].value


16
17
18
19
CPU times: user 1.52 s, sys: 656 ms, total: 2.17 s
Wall time: 39min 38s


In [8]:
[x[0] for x in results]

[0.06079030066607656,
 0.06079030066607656,
 0.06079030066607656,
 0.06079030066607656,
 0.1261645628464289,
 0.1261645628464289,
 0.1261645628464289,
 0.1261645628464289,
 0.5152348482727711,
 0.5152348482727711,
 0.5152348482727711,
 0.5152348482727711,
 0.09641697200995686,
 0.09641697200995686,
 0.09641697200995686,
 0.09641697200995686,
 0.2053082412208569,
 0.2053082412208569,
 0.2053082412208569,
 0.2053082412208569]

In [ ]:
_pag1_res = [x[0] for x in results]
_pag1_genes = [x[1] for x in results]

In [ ]:
import pickle

pickle.dump(pag1_res, open('pag1_res_100_000evals', 'wb'))
pickle.dump(pag1_genes, open('pag1_genes_100_000evals', 'wb'))

In [ ]:
plot_results(data[3][0], _pag1_res)

In [ ]:
plot_results(data[3][0], pag1_res)

# Keijzer 6

In [ ]:
%%time

kei6_res, kei6_genes = run_experiment(data[4], cost_function)

pickle.dump(kei6_res, open('kei6_res_100_000evals', 'wb'))
pickle.dump(kei6_genes, open('kei6_genes_100_000evals', 'wb'))

plot_results(data[4][0], kei6_res)

# Korns 12

In [ ]:
%%time

korns12_res, korns12_genes = run_experiment(data[5], cost_function)

pickle.dump(korns12_res, open('korns12_res_100_000evals', 'wb'))
pickle.dump(korns12_genes, open('korns12_genes_100_000evals', 'wb'))

In [ ]:
%%time

name, (x_train, y_train, x_test, y_test), params = data[5]
print(name)

bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

results = Parallel(n_jobs=num_cores)(delayed(run_experiment_instance)(_, cost_function, x_train, y_train, params, bounds)
                                    for _ in range(100))

In [ ]:
plot_results(data[5][0], korns12_res)

# Vladislasleva 4

In [ ]:
%%time

vlad4_res, vlad4_genes = run_experiment(data[6], cost_function)

pickle.dump(vlad4_res, open('vlad4_res_100_000evals', 'wb'))
pickle.dump(vlad4_genes, open('vlad4_genes_100_000evals', 'wb'))

In [ ]:
plot_results(data[6][0], vlad4_res)

# Rounding of results

In [ ]:
rounded_results = []

chunk = 100
for i, (name, (x_train, y_train, x_test, y_test), params) in zip(range(5), data):
    problem_results = []
    
    x_train = np.c_[x_train, np.ones(len(x_train))]
        
    bounds = tengp.individual.IndividualBuilder(params).create().bounds
    
    print(name)
    
    for genes in champions[chunk*i:chunk*i + chunk]:
        
        individual = params.individual_class(np.round(genes), bounds, params)
        
        output = individual.transform(x_train)
        
        try:
            problem_results.append(mean_squared_error(output, y_train))
        except ValueError:
            print('value error')
    
    rounded_results.append(problem_results)
        
        

In [ ]:
np.min(rounded_results[4])